In [7]:
def adjust_state(state, rotation=0, flip_horizontal=False, flip_vertical=False):
    direction_index = {'UP': 0, 'RIGHT': 1, 'DOWN': 2, 'LEFT': 3}
    directions = ['UP', 'RIGHT', 'DOWN', 'LEFT']  # Define the initial order of directions before rotation

    # Normalize and apply clockwise rotation
    rotation = (rotation // 90) % 4  # Normalize the rotation to one of 0, 1, 2, or 3
    directions = directions[-rotation:] + directions[:-rotation]  # Rotate the list using slicing

    # Handle horizontal flip
    if flip_horizontal:
        directions[1], directions[3] = directions[3], directions[1]  # Swap 'RIGHT' and 'LEFT'

    # Handle vertical flip
    if flip_vertical:
        directions[0], directions[2] = directions[2], directions[0]  # Swap 'UP' and 'DOWN'

    # Update the state according to the new direction list
    new_state = [0] * 6
    for i, dir in enumerate(directions):
        new_state[i] = state[direction_index[dir]]
    new_state[4] = state[4]  # 'CENTER' remains unchanged
    new_state[5] = state[5]  # 'BOMBS_LEFT' remains unchanged

    return new_state

In [8]:
state = ['U', 'R', 'D', 'L', 'C', 'B']

adjust_state(state, 0, True)

['L', 'U', 'R', 'D', 'C', 'B']